# Bước 3: Tổng hợp dữ liệu

Notebook này thực hiện các yêu cầu của Bước 3, dựa trên logic từ file `3_data synthesis.ipynb`.

### 0. Tải thư viện & Thiết lập

In [6]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

print("Bắt đầu Bước 3: Tổng hợp dữ liệu...")

# --- 0. Thiết lập: Định nghĩa đường dẫn ---
try:
    BASE_DIR = os.getcwd()
    PROCESSED_DIR = os.path.join(BASE_DIR, "..", "processed")
    REPORTS_DIR = os.path.join(BASE_DIR, "..", "reports")
    
    if not os.path.exists(PROCESSED_DIR):
        PROCESSED_DIR = "../processed"
        REPORTS_DIR = "../reports"
        
except Exception:
    PROCESSED_DIR = "../processed"
    REPORTS_DIR = "../reports"

print(f"Thư mục PROCESSED: {os.path.abspath(PROCESSED_DIR)}")
print(f"Thư mục REPORTS: {os.path.abspath(REPORTS_DIR)}")

Bắt đầu Bước 3: Tổng hợp dữ liệu...
Thư mục PROCESSED: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\processed
Thư mục REPORTS: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\reports


### 1. Tải dữ liệu đã làm sạch từ Bước 2

In [7]:
micro_clean_path = os.path.join(PROCESSED_DIR, "microblogs_clean.csv")
weather_clean_path = os.path.join(PROCESSED_DIR, "weather_clean.csv")
keywords_clean_path = os.path.join(PROCESSED_DIR, "keywords_cleaned.csv")

try:
    # Quan trọng: parse_dates=['timestamp'] để chuyển cột thời gian về đúng kiểu
    micro_df = pd.read_csv(micro_clean_path, parse_dates=['timestamp'])
    weather_df = pd.read_csv(weather_clean_path, parse_dates=['timestamp'])
    keywords_df = pd.read_csv(keywords_clean_path)
    
    print(f"Tải thành công {len(micro_df)} microblogs đã làm sạch.")
    print(f"Tải thành công {len(weather_df)} bản ghi thời tiết đã làm sạch.")
    print(f"Tải thành công {len(keywords_df)} từ khoá đã làm sạch.")

except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy file ..._clean.csv: {e}")
    print("Vui lòng chạy lại Bước 2.")

display(micro_df.head())
display(weather_df.head())
display(keywords_df.head())

Tải thành công 182422 microblogs đã làm sạch.
Tải thành công 4 bản ghi thời tiết đã làm sạch.
Tải thành công 1301 từ khoá đã làm sạch.


,ID,timestamp,Location,text,text_clean,lat,lon,flag_bad_timestamp,flag_missing_text,flag_missing_location,flag_invalid_location
0,3,2011-05-14 09:22:00,42.22479 93.35922,Nike 'gana' el Mundial al patrocinador oficial...,nike 'gana' el mundial al patrocinador oficial...,42.22479,93.35922,False,False,False,False
1,3,2011-05-12 09:22:00,42.27167 93.42951,adidas vs puma en managment tv. Muy interesante !,adidas vs puma en managment tv. muy interesante !,42.27167,93.42951,False,False,False,False
2,3,2011-05-13 17:21:00,42.2838 93.47745,unsure if I will ever feel comfortable flying ...,unsure if i will ever feel comfortable flying ...,42.28380,93.47745,False,False,False,False
3,4,2011-05-13 07:22:00,42.16877 93.40755,Skinny Cinnamon Dolce Latte from Starbucks ma...,skinny cinnamon dolce latte from starbucks mak...,42.16877,93.40755,False,False,False,False
4,4,2011-05-12 07:22:00,42.21211 93.38398,9. Lemon Cake from starbucks,9. lemon cake from starbucks,42.21211,93.38398,False,False,False,False


,timestamp,Weather,Average_Wind_Speed,Wind_Direction,flag_bad_timestamp,flag_missing_weather,flag_missing_wind_dir,flag_negative_wind_speed,flag_abnormal_wind_speed
0,2011-05-12,CLOUDY,8,N,False,False,False,False,False
1,2011-05-13,RAIN,10,N,False,False,False,False,False
2,2011-05-14,CLOUDY,8,NW,False,False,False,False,False
3,2011-05-15,SUNNY,8,NW,False,False,False,False,False


,keyword
0,0
1,from
2,newtwitter
3,life
4,u


### 2. Yêu cầu 2 (Phần 1): Phân loại từ khoá

**CẬP NHẬT:** Các danh sách dưới đây đã được điền sẵn dựa trên tệp `keywords_cleaned.csv` của bạn. 
Bạn có thể tuỳ chỉnh (thêm/bớt) các từ khoá này nếu muốn.

In [ ]:
# Lấy tất cả từ khoá từ file
try:
    all_keywords = list(keywords_df['keyword'].unique())
except NameError:
    print("LỖI: không thể tải `keywords_df`. Vui lòng chạy lại cell 1.")
    all_keywords = []

# ****************** DANH SÁCH ĐÃ PHÂN LOẠI ******************
# (Dựa trên tệp keywords_cleaned.csv và ví dụ từ đề bài)

symptom_keywords = [
    'feel', 'bad', 'caught', 'chills', 'sick', 'fever', 'flu', 'cold', 
    'head', 'headache', 'breath', 'shortness', 'breathing', 'stomach', 'hurts', 
    'pain', 'pneumonia', 'fatigue', 'sweats', 'diarrhea', 'cough', 'sore', 
    'throat', 'hurt', 'dry', 'suffering', 'dies', 'death', 'sickly', 
    'dreadful', 'aching', 'cry', 'appetite', 'burning', 'muscles', 'runny', 
    'bedridden', 'scary', 'mess', 'crying', 'rough', 'dying', 'lazy', 
    'suck', 'ruined', 'shock', 'killed', 'trouble', 'difficult', 'ill', 
    'body', 'chest', 'scared',
    
    'medicine', 'doctor', 'hospital', 'nurse', 'health', 'crisis', 
    'disaster', 'help', 'water', 'air', 'therapy', 'drug', 'dr', 'pray'
]

other_keywords = [
    'medicine', 'doctor', 'hospital', 'nurse', 'health', 'crisis', 
    'disaster', 'help', 'water', 'air', 'therapy', 'drug', 'dr', 'pray'
]
# ************************************************************

# --- Tự động lọc lại --- 
# Đảm bảo các từ bạn chọn có trong danh sách gốc
symptom_keywords_final = [k for k in symptom_keywords if k in all_keywords]
other_keywords_final = [k for k in other_keywords if k in all_keywords]

# Kiểm tra xem có từ nào bị lọc không (do gõ sai)
missing_symptoms = set(symptom_keywords) - set(symptom_keywords_final)
if missing_symptoms:
    print(f"Cảnh báo: Các từ triệu chứng này không có trong tệp: {missing_symptoms}")

print(f"Đã phân loại {len(symptom_keywords_final)} từ khoá triệu chứng.")
print(f"Đã phân loại {len(other_keywords_final)} từ khoá khác.")

# Lưu tệp processed/keywords.csv theo yêu cầu
df_keywords_processed = pd.DataFrame({
    'symptom_keyword': pd.Series(symptom_keywords_final),
    'other_keyword': pd.Series(other_keywords_final)
})
keywords_output_path = os.path.join(PROCESSED_DIR, "keywords.csv")
df_keywords_processed.to_csv(keywords_output_path, index=False, encoding='utf-8')
print(f"Đã lưu danh sách từ khoá đã phân loại vào: {keywords_output_path}")

display(df_keywords_processed.head())

Đã phân loại 67 từ khoá triệu chứng.
Đã phân loại 14 từ khoá khác.
Đã lưu danh sách từ khoá đã phân loại vào: c:\Users\admin\OneDrive\Máy tính\LTXLDL\2\2526-LTXLDL-Project-2.4\src\..\processed\keywords.csv


,symptom_keyword,other_keyword
0,feel,medicine
1,bad,doctor
2,caught,hospital
3,chills,nurse
4,sick,health


### 3. Yêu cầu 1: Nội suy (upsample) và Kết nối (join)

Thực hiện nội suy dữ liệu thời tiết (từ hàng ngày -> hàng giờ) và gộp vào bảng microblogs.

In [9]:
print("\nĐang thực hiện Yêu cầu 1: Nội suy và kết nối dữ liệu...")

# A. Nội suy thời tiết lên theo giờ
weather_sub = weather_df.set_index('timestamp')
# Resample theo giờ ('h') và điền tiếp giá trị cũ ('ffill' - forward fill)
weather_hourly = weather_sub.resample('h').ffill()
weather_hourly_sorted = weather_hourly.reset_index().sort_values('timestamp')

# B. Sắp xếp microblogs theo thời gian
micro_sub = micro_df.sort_values('timestamp')

# C. Kết nối (merge_asof)
# Tìm bản ghi thời tiết gần nhất (nhưng không muộn hơn) cho mỗi microblog
df_merged = pd.merge_asof(
    micro_sub,
    weather_hourly_sorted[['timestamp', 'Weather', 'Wind_Direction', 'Average_Wind_Speed']],
    on='timestamp',
    direction='backward' # 'backward' nghĩa là tìm giá trị trước đó
)

# Đổi tên cột cho rõ ràng
df_merged = df_merged.rename(columns={
    'Weather': 'weather',
    'Wind_Direction': 'wind_direction',
    'Average_Wind_Speed': 'average_wind_speed'
})

print(f"Kết nối thành công. Bảng microblogs đã có {len(df_merged.columns)} cột (bao gồm cả thời tiết).")
display(df_merged[['timestamp', 'text_clean', 'weather', 'wind_direction']].head())


Đang thực hiện Yêu cầu 1: Nội suy và kết nối dữ liệu...
Kết nối thành công. Bảng microblogs đã có 14 cột (bao gồm cả thời tiết).


,timestamp,text_clean,weather,wind_direction
0,2011-05-12,is there a shelfari for films?,CLOUDY,N
1,2011-05-12,ops! opa! #newtwitter #modeon,CLOUDY,N
2,2011-05-12,the sgs notifies of available wifi networks ev...,CLOUDY,N
3,2011-05-12,monday! kussh on dekc fire 2,CLOUDY,N
4,2011-05-12,#newtwitter means much more to me since i've a...,CLOUDY,N


### 4. Yêu cầu 2 & 3: Tìm từ khoá và "Explode"

Tìm các từ khoá triệu chứng trong mỗi blog và "explode" (tách) ra để mỗi từ khoá nằm trên một hàng.

In [10]:
print("\nĐang thực hiện Yêu cầu 2 & 3: Tìm kiếm và thống kê từ khoá...")

# A. Hàm tìm từ khoá (lấy từ file mẫu)
def find_keywords(text, keywords):
    # Tạo regex \b(word1|word2|word3)\b để tìm các từ riêng biệt
    # Dùng re.escape để xử lý các ký tự đặc biệt (nếu có)
    pattern = r'\b(' + '|'.join(re.escape(k) for k in keywords) + r')\b'
    # re.findall sẽ trả về một list các từ tìm thấy
    found = re.findall(pattern, str(text), re.IGNORECASE) # Thêm str(text) để tránh lỗi
    # Trả về list duy nhất (unique)
    return list(set(found))

# B. Áp dụng hàm tìm kiếm cho *chỉ* các từ khoá triệu chứng
print("Đang tìm các từ khoá triệu chứng trong microblogs...")
df_merged['symptom_keywords_found'] = df_merged['text_clean'].apply(
    lambda text: find_keywords(text, symptom_keywords_final) # Dùng list_final
)

# C. "Explode" DataFrame
# Lọc ra các blogs có tìm thấy ít nhất 1 từ khoá
df_keywords_expanded = df_merged[df_merged['symptom_keywords_found'].map(len) > 0].copy()
# "Explode" để mỗi từ khoá trên một hàng
df_keywords_expanded = df_keywords_expanded.explode('symptom_keywords_found')
df_keywords_expanded = df_keywords_expanded.rename(columns={'symptom_keywords_found': 'keyword'})

# D. Thêm cột 'hour' để groupby
df_keywords_expanded['hour'] = df_keywords_expanded['timestamp'].dt.floor('h')

print(f"Đã \"explode\" dữ liệu, có {len(df_keywords_expanded)} lượt từ khoá triệu chứng xuất hiện.")
display(df_keywords_expanded[['timestamp', 'keyword', 'hour']].head())


Đang thực hiện Yêu cầu 2 & 3: Tìm kiếm và thống kê từ khoá...
Đang tìm các từ khoá triệu chứng trong microblogs...
Đã "explode" dữ liệu, có 20736 lượt từ khoá triệu chứng xuất hiện.


,timestamp,keyword,hour
8,2011-05-12 00:00:00,help,2011-05-12
12,2011-05-12 00:00:00,crisis,2011-05-12
13,2011-05-12 00:00:00,crisis,2011-05-12
34,2011-05-12 00:00:00,pain,2011-05-12
49,2011-05-12 00:01:00,bad,2011-05-12


### 5. Yêu cầu 2: Lưu `stat_hourly_HH_DD_MM.csv`

Thống kê số lần xuất hiện, thời tiết phổ biến, hướng gió phổ biến cho mỗi từ khoá theo giờ.

In [11]:
print("\nĐang lưu thống kê Yêu cầu 2 (stat_hourly)...")

# Hàm lấy mode (giá trị phổ biến nhất), (lấy từ file mẫu)
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

# Groupby theo giờ và từ khoá, sau đó tổng hợp
hourly_stats = df_keywords_expanded.groupby(['hour', 'keyword']).agg(
    count=pd.NamedAgg(column='keyword', aggfunc='size'),
    weather=pd.NamedAgg(column='weather', aggfunc=get_mode),
    wind_direction=pd.NamedAgg(column='wind_direction', aggfunc=get_mode)
).reset_index()

# D. Lặp qua từng giờ và lưu file
count_files_stat = 0
for hour, data in hourly_stats.groupby('hour'):
    # Định dạng tên file: HH_DD_MM
    filename = f"stat_hourly_{hour.strftime('%H_%d_%m')}.csv"
    output_path = os.path.join(PROCESSED_DIR, filename)
    
    # Chỉ lưu dữ liệu của giờ đó
    data.drop(columns=['hour']).to_csv(output_path, index=False, encoding='utf-8')
    count_files_stat += 1

print(f"Đã lưu {count_files_stat} tệp 'stat_hourly_...'.")
display(hourly_stats.head())


Đang lưu thống kê Yêu cầu 2 (stat_hourly)...
Đã lưu 96 tệp 'stat_hourly_...'.


,hour,keyword,count,weather,wind_direction
0,2011-05-12,air,6,CLOUDY,N
1,2011-05-12,appetite,1,CLOUDY,N
2,2011-05-12,bad,25,CLOUDY,N
3,2011-05-12,body,4,CLOUDY,N
4,2011-05-12,burning,1,CLOUDY,N


### 6. Yêu cầu 3: Lưu `keyword_location_mapping_hourly_HH_DD_MM.csv`

Thống kê các vị trí (lat, lon) nơi mỗi từ khoá xuất hiện theo giờ.

In [12]:
print("\nĐang lưu thống kê Yêu cầu 3 (keyword_location_mapping)...")

# Groupby theo giờ, từ khoá, và vị trí (Bỏ qua các hàng thiếu vị trí)
location_mapping_raw = df_keywords_expanded.dropna(subset=['lat', 'lon'])

location_mapping = location_mapping_raw.groupby(['hour', 'lat', 'lon', 'keyword']).size().reset_index(name='count')

# Chỉ giữ các cột cần thiết
location_mapping = location_mapping[['hour', 'lat', 'lon', 'keyword']]

# Tạo cột 'location' từ 'lat' và 'lon'
location_mapping['location'] = location_mapping['lat'].astype(str) + ' ' + location_mapping['lon'].astype(str)

# D. Lặp qua từng giờ và lưu file
count_files_loc = 0
for hour, data in location_mapping.groupby('hour'):
    # Định dạng tên file
    filename = f"keyword_location_mapping_hourly_{hour.strftime('%H_%d_%m')}.csv"
    output_path = os.path.join(PROCESSED_DIR, filename)
    
    # Chỉ lưu các cột 'location' và 'keyword' theo yêu cầu
    data[['location', 'keyword']].to_csv(output_path, index=False, encoding='utf-8')
    count_files_loc += 1

print(f"Đã lưu {count_files_loc} tệp 'keyword_location_mapping_...'.")
print("\n--- Bước 3: Hoàn thành ---")
display(location_mapping.head())


Đang lưu thống kê Yêu cầu 3 (keyword_location_mapping)...
Đã lưu 96 tệp 'keyword_location_mapping_...'.

--- Bước 3: Hoàn thành ---


,hour,lat,lon,keyword,location
0,2011-05-12,42.16287,93.47385,crisis,42.16287 93.47385
1,2011-05-12,42.16316,93.49462,crisis,42.16316 93.49462
2,2011-05-12,42.16582,93.42712,feel,42.16582 93.42712
3,2011-05-12,42.16818,93.43830,crisis,42.16818 93.4383
4,2011-05-12,42.17260,93.28493,headache,42.1726 93.28493
